Получаем данные о корпоративных событиях. Из них для начала надо выбрать данные по датам выхода МСФО отчетности и связать их с котировками и самой отчетностью. 

In [ ]:
import requests
import pandas as pd
import time

# Базовый URL для API
base_url = "https://financemarker.ru/api/fm/v2/disclosure"
api_token = ""  # Ваш API-токен
limit = 100  # Количество записей на запрос
initial_offset = 200  # Начальный offset

# Пустой DataFrame для хранения всех данных
all_data = pd.DataFrame()

# Цикл для уменьшения offset на 100 с каждым запросом
for offset in range(initial_offset, -1, -100):
    # Параметры запроса
    params = {
        'limit': limit,
        'offset': offset,
        'sort_by': 'date',
        'sort_order': 'DESC',
        'api_token': api_token
    }
    
    # Выполняем запрос к API
    response = requests.get(base_url, params=params)
    
    # Проверяем, успешно ли выполнен запрос
    if response.status_code == 200:
        data = response.json()
        # Выводим структуру данных для отладки
        print(f"Структура данных при offset={offset}: {type(data)}")
        
        # Проверка, является ли data списком
        if isinstance(data, list):
            if len(data) > 0:
                # Преобразуем данные в DataFrame
                df = pd.DataFrame(data)
                # Добавляем данные в общий DataFrame
                all_data = pd.concat([all_data, df], ignore_index=True)
                print(f"Данные с offset={offset} успешно добавлены. Всего записей: {len(all_data)}")
            else:
                print(f"Пустые данные с offset={offset}, прекращаем парсинг.")
                break
        else:
            print(f"Неожиданный формат данных при offset={offset}: {data}")
            break
    else:
        print(f"Ошибка при запросе данных с offset={offset}: {response.status_code}")
        break
    
    # Пауза между запросами, чтобы избежать блокировки
    time.sleep(1)

# Сохраняем итоговый DataFrame в Excel
all_data.to_excel('new_data.xlsx', index=False)

print("Все данные успешно собраны и сохранены в файле financemarker_data.xlsx")

Структура данных при offset=200: <class 'list'>
Данные с offset=200 успешно добавлены. Всего записей: 100
Структура данных при offset=100: <class 'list'>
Данные с offset=100 успешно добавлены. Всего записей: 200
Структура данных при offset=0: <class 'list'>
Данные с offset=0 успешно добавлены. Всего записей: 300
Все данные успешно собраны и сохранены в файле financemarker_data.xlsx


In [143]:
df1 = pd.read_excel('disclosure_data.xlsx')
df2 = pd.read_excel('new_data.xlsx')
final_data = pd.concat([df1, df2], ignore_index=True)
final_data.drop_duplicates(inplace=True)
final_data.to_excel('disclosure_data.xlsx', index=False)

Для примера возьмем только отчеты и отчеты только по МСФО по тикерам входящим в состав индекса IMOEX

In [144]:
filtered_df = final_data[final_data['category'] == 'report']
filtered_df = filtered_df[filtered_df['type'] == 'МСФО']
imoex_base = pd.read_csv('imoex_base.csv') 
filtered_df = filtered_df[filtered_df['code'].isin(imoex_base['Ticker'])]

Теперь получаем всю информацию доступную информацию по тикерам из IMOEX

In [ ]:
unique_codes = imoex_base['Ticker'].unique()

# Основной URL для запроса
url_template = "https://financemarker.ru/api/fm/v2/stocks/MOEX:{code}"

# Параметры запроса
params = {
    'include': 'ratios,reports,dividends,ideas,ideasAggregated,info,insiderTransactions,owners,shares,summary',
    'api_token': api_token  # Используйте ваш актуальный API токен
}

# Заголовки запроса
headers = {
    'accept': 'application/json'
}

# Создание пустых словарей для хранения данных каждого блока
ratios_data = []
reports_data = []
dividends_data = []
ideas_data = []
ideas_aggregated_data = []
info_data = []
insider_transactions_data = []
owners_data = []
shares_data = []
summary_data = []

# Функция для добавления данных в соответствующий список
def add_data(block_name, data, code):
    for item in data:
        if item is not None and isinstance(item, dict): 
            item['code'] = code  # Добавляем код компании к данным
            if block_name == 'ratios':
                ratios_data.append(item)
            elif block_name == 'reports':
                reports_data.append(item)
            elif block_name == 'dividends':
                dividends_data.append(item)
            elif block_name == 'ideas':
                ideas_data.append(item)
            elif block_name == 'ideasAggregated':
                ideas_aggregated_data.append(item)
            elif block_name == 'info':
                info_data.append(item)
            elif block_name == 'insiderTransactions':
                insider_transactions_data.append(item)
            elif block_name == 'owners':
                owners_data.append(item)
            elif block_name == 'shares':
                shares_data.append(item)
            elif block_name == 'summary':
                summary_data.append(item)

# Выполнение запроса для каждого уникального кода
for code in unique_codes:
    # Формирование URL с подстановкой кода
    url = url_template.format(code=code)
    
    # Выполнение запроса
    response = requests.get(url, params=params, headers=headers)
    
    # Проверка статуса ответа
    if response.status_code == 200:
        data = response.json()
        
        # Добавление данных в соответствующие списки
        for block_name, block_data in data.items():
            if isinstance(block_data, list):
                add_data(block_name, block_data, code)
            else:
                add_data(block_name, [block_data], code)  # Если данные не в виде списка
        
        print(f"Данные для {code} успешно получены.")
    else:
        print(f"Ошибка при запросе данных для {code}: {response.status_code}")

# Создание DataFrame для каждого блока
ratios_df = pd.DataFrame(ratios_data)
reports_df = pd.DataFrame(reports_data)
dividends_df = pd.DataFrame(dividends_data)
ideas_df = pd.DataFrame(ideas_data)
ideas_aggregated_df = pd.DataFrame(ideas_aggregated_data)
info_df = pd.DataFrame(info_data)
insider_transactions_df = pd.DataFrame(insider_transactions_data)
owners_df = pd.DataFrame(owners_data)
shares_df = pd.DataFrame(shares_data)
summary_df = pd.DataFrame(summary_data)

# Словарь для хранения DataFrame и соответствующих названий файлов
dataframes = {
    'ratios': ratios_df,
    'reports': reports_df,
    'dividends': dividends_df,
    'ideas': ideas_df,
    'ideasAggregated': ideas_aggregated_df,
    'info': info_df,
    'insiderTransactions': insider_transactions_df,
    'owners': owners_df,
    'shares': shares_df,
    'summary': summary_df
}

# Сохранение каждого DataFrame в отдельный Excel файл
file_paths = {}
for name, df in dataframes.items():
    file_name = f"{name}.xlsx"
    df.to_excel(file_name, index=False)
    file_paths[name] = file_name

Данные для AFKS успешно получены.
Данные для AFLT успешно получены.
Данные для AGRO успешно получены.
Данные для ALRS успешно получены.
Данные для ASTR успешно получены.
Данные для BSPB успешно получены.
Данные для CBOM успешно получены.
Данные для CHMF успешно получены.
Данные для ENPG успешно получены.
Данные для FEES успешно получены.
Данные для FIVE успешно получены.
Данные для FLOT успешно получены.
Данные для GAZP успешно получены.
Данные для GMKN успешно получены.
Данные для HYDR успешно получены.
Данные для IRAO успешно получены.
Данные для LEAS успешно получены.
Данные для LKOH успешно получены.
Данные для MAGN успешно получены.
Данные для MGNT успешно получены.
Данные для MOEX успешно получены.
Данные для MSNG успешно получены.
Данные для MTLR успешно получены.
Данные для MTLRP успешно получены.
Данные для MTSS успешно получены.
Данные для NLMK успешно получены.
Данные для NVTK успешно получены.
Данные для OZON успешно получены.
Данные для PHOR успешно получены.
Данные для PI

Теперь скачиваем котировки акций по API MOEX

In [146]:
import backtrader as bt
import pandas as pd
import requests
from datetime import datetime, timedelta

# Формирование дат начала и конца периода
end_date = datetime.now()
start_date = end_date - timedelta(days=7000)

# Преобразование дат в строковый формат
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Инициализация переменных для цикла
start_index = 0
batch_size = 500  # предполагаемое ограничение количества записей на один запрос
all_candles_data = []  # список для хранения всех полученных данных
for code in unique_codes:
    while True:
        # URL для запроса данных с учетом start_index
        url = f'http://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{code}/candles.json?from={start_date_str}&till={end_date_str}&interval=24&start={start_index}'

        # Выполнение запроса
        response = requests.get(url)
        data = response.json()

        # Извлечение данных свечек
        candles_data = data['candles']['data']

        # Проверка наличия данных в ответе
        if not candles_data:
            break  # Выход из цикла, если данных больше нет

        # Добавление полученных данных в общий список
        all_candles_data.extend(candles_data)

        # Увеличение start_index для следующего запроса
        start_index += batch_size

    # Создание DataFrame из всех полученных данных
    df = pd.DataFrame(all_candles_data, columns=data['candles']['columns'])

    # Преобразование типов данных для даты и времени
    df['begin'] = pd.to_datetime(df['begin'])

    df.set_index('begin', inplace=True)

    # Переименование столбцов
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})

    # Удаление ненужных столбцов
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    df.to_excel(f'{code}.xlsx')
    

Для примера будем работать с reports.xlsx

In [147]:
reports_df = pd.read_excel('reports.xlsx')
reports_df.columns
# Список столбцов для конвертации
columns_to_convert = [
    'accounts_payable', 'accounts_payable_current', 'accounts_payable_long', 
    'accounts_receivable', 'accounts_receivable_current', 'accounts_receivable_long', 
    'amount', 'capex', 'cash_and_equiv', 'cash_paid_for_interest', 'cff', 'cfi', 'cfo', 
    'commission_expense', 'commission_income', 'commission_net', 'cost_of_sales', 
    'current_assets', 'current_debt', 'current_lease', 'current_liabilities', 
    'depr_depl_amort', 'earnings', 'earnings_stock_holders', 'earnings_wo_tax', 'ebit', 
    'ebitda', 'equity', 'equity_stock_holders', 'fcf', 'ffo', 'goodwill', 'gross_profit', 
    'interest_expense', 'interest_income', 'interest_net', 'inventories', 'issuance_of_debt', 
    'long_term_assets', 'long_term_debt', 'long_term_investments', 'long_term_lease', 
    'long_term_liabilities', 'month', 'net_debt', 'net_issuance_of_debt', 'operating_income', 
    'payments_for_dividends', 'payments_of_debt', 'property_plant_equipment', 
    'repurchase_of_stock', 'research_and_development', 'retained_earnings', 'revenue', 
    'sel_gen_adm_expenses', 'share_premium', 'total_assets', 'total_debt', 'total_expenses', 
    'total_liabilities', 'year'
]

# Проверка наличия всех столбцов в DataFrame
valid_columns = [col for col in columns_to_convert if col in reports_df.columns]

# Вывод списка валидных столбцов, которые будут конвертированы
print(f"Столбцы, которые будут конвертированы: {valid_columns}")

# Преобразование указанных столбцов в числовой формат
reports_df[valid_columns] = reports_df[valid_columns].apply(pd.to_numeric, errors='coerce')

Столбцы, которые будут конвертированы: ['accounts_payable', 'accounts_payable_current', 'accounts_payable_long', 'accounts_receivable', 'accounts_receivable_current', 'accounts_receivable_long', 'amount', 'capex', 'cash_and_equiv', 'cash_paid_for_interest', 'cff', 'cfi', 'cfo', 'commission_expense', 'commission_income', 'commission_net', 'cost_of_sales', 'current_assets', 'current_debt', 'current_lease', 'current_liabilities', 'depr_depl_amort', 'earnings', 'earnings_stock_holders', 'earnings_wo_tax', 'ebit', 'ebitda', 'equity', 'equity_stock_holders', 'fcf', 'ffo', 'goodwill', 'gross_profit', 'interest_expense', 'interest_income', 'interest_net', 'inventories', 'issuance_of_debt', 'long_term_assets', 'long_term_debt', 'long_term_investments', 'long_term_lease', 'long_term_liabilities', 'month', 'net_debt', 'net_issuance_of_debt', 'operating_income', 'payments_for_dividends', 'payments_of_debt', 'property_plant_equipment', 'repurchase_of_stock', 'research_and_development', 'retained_ea

Для простоты оставляем только годовые отчеты по МСФО

In [148]:
reports_df = reports_df[reports_df['type'] == 'МСФО']
reports_df = reports_df[reports_df['period'] == 'Y']

numeric_columns = reports_df.select_dtypes(include='number').columns.tolist()

# Умножаем все числовые столбцы на столбец 'amount' (это число на которое надо умножить данные из отчетов)
for col in numeric_columns:
    if col not in ('year', 'month'):
        reports_df[col] = reports_df[col] * reports_df['amount']
        
disclosure_df = filtered_df[filtered_df['period'] == 'Y']
disclosure_df = filtered_df[filtered_df['period'] == 'Y']

Соеденяме информацию из отчетов с их датами выхода

In [149]:
# Объединяем DataFrame по колонкам 'year' и 'code'
merged_df = reports_df.merge(disclosure_df[['year', 'code', 'date', 'link']], on=['year', 'code'], how='left')
reports_df = merged_df.copy()

Теперь добавим признак пропуска данных в датасет

In [150]:
for column in reports_df.columns:
    reports_df[f'{column}_missing'] = reports_df[column].isnull().astype(int)

Проставляем даты выхода отчета, там где их нет. Если есть пропуск то берем максимальный месяц + день выхода и прибовляем 1 год к отчетному году.

In [151]:
# Преобразуем даты в datetime
reports_df['date'] = pd.to_datetime(reports_df['date'], errors='coerce')

# Находим самый поздний день и месяц для каждого `code`
max_day_month = reports_df.groupby('code')['date'].agg(
    lambda x: x.dropna().max()  # Находим самую позднюю дату
).reset_index()

# Извлекаем день и месяц из самой поздней даты
max_day_month['max_day'] = max_day_month['date'].dt.day.fillna(1).astype(int)  # Заполняем NaN значением 1
max_day_month['max_month'] = max_day_month['date'].dt.month.fillna(1).astype(int)  # Заполняем NaN значением 1

# Объединяем эту информацию с исходным DataFrame
reports_df = reports_df.merge(max_day_month[['code', 'max_day', 'max_month']], on='code', how='left')

# Заполняем пропущенные даты
def fill_missing_dates(row):
    if pd.isnull(row['date']):  # Если дата отсутствует
        return pd.Timestamp(
            year=int(row['year'] + 1), 
            month=int(row['max_month']), 
            day=int(row['max_day'])
        )
    return row['date']  # Если дата есть, оставляем как есть

reports_df['date'] = reports_df.apply(fill_missing_dates, axis=1)

# Удаляем временные столбцы, если они больше не нужны
reports_df.drop(['max_day', 'max_month'], axis=1, inplace=True)

Объеденяем котировки по всем тикерам

In [152]:
import pandas as pd
import os

# Список тикеров
final_tickers = reports_df['code'].unique()

# Путь к файлам с котировками (в корне проекта)
path_to_quotes = "./"

# Загружаем только те файлы, которые соответствуют тикерам
quotes = []
for ticker in final_tickers:
    file_path = os.path.join(path_to_quotes, f"{ticker}.xlsx")
    if os.path.exists(file_path):
        df = pd.read_excel(file_path)
        df['code'] = ticker  # Добавляем тикер как отдельный столбец
        quotes.append(df)

# Объединяем все котировки в один DataFrame
quotes_df = pd.concat(quotes, ignore_index=True)
quotes_df['begin'] = pd.to_datetime(quotes_df['begin'])  # Преобразуем даты

# Устанавливаем мультииндекс
# quotes_df.set_index(['begin', 'code'], inplace=True)

# Проверяем результат
print(quotes_df.head())

       begin    Open    High     Low   Close    Volume  code
0 2014-06-09  44.364  45.001  43.751  44.448   4380200  AFKS
1 2014-06-10  44.440  45.596  44.117  45.499  11586400  AFKS
2 2014-06-11  45.007  45.749  44.700  45.300   4757700  AFKS
3 2014-06-16  45.913  46.370  44.514  45.999  17932600  AFKS
4 2014-06-17  46.300  46.467  45.700  46.100   5544500  AFKS


In [153]:
reports_df['date'] = pd.to_datetime(reports_df['date'])

# Добавляем "end_date" (до какой даты действует отчет)
reports_df['end_date'] = reports_df.groupby('code')['date'].shift(-1)
reports_df['end_date'] = reports_df['end_date'].fillna(pd.Timestamp('2099-12-31'))  # Для последнего отчета

# Джойним котировки с отчетами
merged_df = pd.merge(
    quotes_df,
    reports_df,
    on='code',
    how='left'
)

# Условие: оставляем только строки, где дата попадает в диапазон [date, end_date]
merged_df = merged_df[
    (merged_df['begin'] >= merged_df['date']) & (merged_df['begin'] < merged_df['end_date'])
]

# Убираем ненужные столбцы
final_df = merged_df.drop(columns=['end_date'])  # Убираем только служебный столбец end_date

# Результат
print(final_df.head(20))  # Покажем первые 20 строк

         begin    Open    High     Low   Close    Volume  code  \
2   2014-06-09  44.364  45.001  43.751  44.448   4380200  AFKS   
15  2014-06-10  44.440  45.596  44.117  45.499  11586400  AFKS   
28  2014-06-11  45.007  45.749  44.700  45.300   4757700  AFKS   
41  2014-06-16  45.913  46.370  44.514  45.999  17932600  AFKS   
54  2014-06-17  46.300  46.467  45.700  46.100   5544500  AFKS   
67  2014-06-18  45.823  46.677  45.823  46.600   6390200  AFKS   
80  2014-06-19  46.683  47.049  46.113  46.599   2484400  AFKS   
93  2014-06-20  46.743  46.763  45.260  46.000   4541800  AFKS   
106 2014-06-23  46.091  46.982  45.030  45.500   5511100  AFKS   
119 2014-06-24  45.582  46.450  45.261  46.450   8348800  AFKS   
132 2014-06-25  46.308  47.332  44.849  45.100   6537800  AFKS   
145 2014-06-26  45.497  45.497  44.137  45.200   2746400  AFKS   
158 2014-06-27  45.200  46.200  44.623  46.200   4036800  AFKS   
171 2014-06-30  46.199  46.338  45.518  45.600   6177400  AFKS   
184 2014-0

In [154]:
final_df.set_index(['begin', 'code'], inplace=True)

In [155]:
final_df

,,Open,High,Low,Close,Volume,accounts_payable,accounts_payable_current,accounts_payable_long,accounts_receivable,accounts_receivable_current,...,sel_gen_adm_expenses_missing,share_premium_missing,total_assets_missing,total_debt_missing,total_expenses_missing,total_liabilities_missing,type_missing,year_missing,date_missing,link_y_missing
begin,code,,,,,,,,,,,,,,,,,,,,,
2014-06-09,AFKS,44.364,45.001,43.751,44.448,4380200,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,1
2014-06-10,AFKS,44.440,45.596,44.117,45.499,11586400,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,1
2014-06-11,AFKS,45.007,45.749,44.700,45.300,4757700,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,1
2014-06-16,AFKS,45.913,46.370,44.514,45.999,17932600,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,1
2014-06-17,AFKS,46.300,46.467,45.700,46.100,5544500,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11,YDEX,7205.500,7235.000,6995.000,7022.500,1556463,1.688650e+11,1.688650e+11,NaN,8.503600e+10,8.503600e+10,...,0,1,0,0,0,0,0,0,0,0
2024-11-12,YDEX,7025.000,7071.500,6967.000,6990.000,691473,1.688650e+11,1.688650e+11,NaN,8.503600e+10,8.503600e+10,...,0,1,0,0,0,0,0,0,0,0
2024-11-13,YDEX,6990.000,7048.000,6918.000,6925.000,739997,1.688650e+11,1.688650e+11,NaN,8.503600e+10,8.503600e+10,...,0,1,0,0,0,0,0,0,0,0
